In [ ]:
pip install bokeh pandas sqlalchemy cassandra-driver


In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, WMTSTileSource, HoverTool
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Configure the connection
auth_provider = PlainTextAuthProvider(username='admin', password='admin')
cluster = Cluster(['scylladb'], auth_provider=auth_provider)
session = cluster.connect()

# Query the data
query = "SELECT callsign, longitude, latitude, on_ground, squawk FROM demo.latest_flight_data"
rows = session.execute(query)

# Convert to pandas DataFrame
data = [{
    "callsign": row.callsign,
    "longitude": row.longitude,
    "latitude": row.latitude,
    "on_ground": row.on_ground,
    "squawk": row.squawk
} for row in rows]
df = pd.DataFrame(data)

# Close the connection
cluster.shutdown()

def wgs84_to_web_mercator(df, lon="longitude", lat="latitude"):
    if df.empty:
        return df  # Return the empty DataFrame to avoid processing further
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

# Check if DataFrame is not empty before transformation
if not df.empty:
    df = wgs84_to_web_mercator(df)

if not df.empty:
    # Set up the Bokeh plot and tile source
    tile_url = "https://tile.openstreetmap.org/{Z}/{X}/{Y}.png"
    tile_source = WMTSTileSource(url=tile_url, attribution=(
        "Map data (c) <a href='https://www.openstreetmap.org/copyright'>"
        "OpenStreetMap contributors</a>"))

    p = figure(title='Aircraft Over Boston', width=800, height=800, x_axis_type="mercator", y_axis_type="mercator", tools="pan,wheel_zoom,reset")
    p.add_tile(tile_source)

    # Set up data source for Bokeh plotting
    source = ColumnDataSource(data=dict(
        x=df['x'],
        y=df['y'],
        callsign=df['callsign'],
        latitude=df['latitude'],  # Added for hover tool
        longitude=df['longitude']  # Added for hover tool
    ))

    # Add circles with hover tool
    p.circle(x='x', y='y', source=source, color='red', size=10)
    hover = HoverTool(tooltips=[
        ("Callsign", "@callsign"),
        ("Lat/Long", "@latitude, @longitude")
    ])
    p.add_tools(hover)

    # Display the plot
    output_notebook()
    show(p)
else:
    print("No data available to plot.")

